In [4]:
    ply_path = r"/home/ws/git/python_motion_planning/point_cloud/datas/total.ply"  # Replace with your PLY file
    cell_size = 1  # 10cm grid cells
    dpi = 200
    dpi = 1000
    dpi = 500
    max_height_var = 1  # 5cm max variation
    max_height_var = 0.3  # 5cm max variation
    max_height_var = 0.5  # 5cm max variation
    max_height_var = 1.0  # 5cm max variation

In [2]:
import math

import numpy as np
import matplotlib.pyplot as plt
from plyfile import PlyData
from matplotlib.colors import ListedColormap

In [5]:
    plydata = PlyData.read(ply_path)
    vertices = plydata['vertex']
    x = vertices['x']
    y = vertices['y']  # Height/attitude values
    z = vertices['z']

    xx = vertices.data['x']

    z = [-i for i in z]

    # Calculate grid dimensions
    min_x, max_x = np.min(x), np.max(x)
    min_z, max_z = np.min(z), np.max(z)
    min_y, max_y = np.min(y), np.max(y)

    range_x = (max_x - min_x)
    range_y = (max_y - min_y)
    range_z = (max_z - min_z)

    grid_size_x = range_x/(math.sqrt(dpi*dpi/(range_x*range_z))*range_x)
    grid_size_z = range_z/(math.sqrt(dpi*dpi/(range_x*range_z))*range_z)



    # grid_size = int((max_x - min_x) / dpi)

    # Create grid bins
    x_bins = np.arange(min_x, max_x + grid_size_x, grid_size_x)
    z_bins = np.arange(min_z, max_z + grid_size_z, grid_size_z)

    # Initialize height extremes
    grid_max = np.full((len(x_bins) - 1, len(z_bins) - 1), -np.inf)
    grid_min = np.full((len(x_bins) - 1, len(z_bins) - 1), np.inf)

    point_count = np.zeros((len(x_bins) - 1, len(z_bins) - 1), dtype=int)
    height_sum = np.zeros((len(x_bins) - 1, len(z_bins) - 1))
    height_avg = np.zeros((len(x_bins) - 1, len(z_bins) - 1), dtype=int)

In [ ]:
    for xi, yi, zi in zip(x, y, z):
        x_idx = np.digitize(xi, x_bins) - 1
        z_idx = np.digitize(zi, z_bins) - 1

        # Handle edge cases
        x_idx = max(0, min(x_idx, grid_max.shape[0] - 1))
        z_idx = max(0, min(z_idx, grid_max.shape[1] - 1))

        # Update height extremes
        grid_max[x_idx, z_idx] = max(grid_max[x_idx, z_idx], yi)
        grid_min[x_idx, z_idx] = min(grid_min[x_idx, z_idx], yi)

        point_count[x_idx, z_idx] += 1
        height_sum[x_idx, z_idx] += yi

    # Calculate height variation per cell
    height_var = grid_max - grid_min